In [1]:
import cv2
import sys
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from paddleocr import PaddleOCR, draw_ocr
import pytesseract
import pandas as pd
import sys

In [2]:
def findTextRegion(img):
    region = []
 
    # 1. 查找轮廓
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    # 2. 筛选那些面积小的
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算该轮廓的面积
        area = cv2.contourArea(cnt) 
 
        # 面积小的都筛选掉
        if(area < 1000):
            continue
 
        # 轮廓近似，作用很小
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
 
        # 找到最小的矩形，该矩形可能有方向
        rect = cv2.minAreaRect(cnt)
        #print "rect is: "
        #print rect
 
        # box是四个点的坐标
        box = cv2.boxPoints(rect)
        box = np.int0(box)
 
        # 计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
 
        # 筛选那些太细的矩形，留下扁的
        if(height > width * 1.2):
            continue
 
        region.append(box)
 
    return region

In [3]:
def preprocess(gray):
    # 1. Sobel算子，x方向求梯度
    sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
    # 2. 二值化
    ret, binary = cv2.threshold(sobel, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
 
    # 3. 膨胀和腐蚀操作的核函数
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 9))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (24, 6))
 
    # 4. 膨胀一次，让轮廓突出
    dilation = cv2.dilate(binary, element2, iterations = 1)
 
    # 5. 腐蚀一次，去掉细节，如表格线等。注意这里去掉的是竖直的线
    erosion = cv2.erode(dilation, element1, iterations = 1)
 
    # 6. 再次膨胀，让轮廓明显一些
    dilation2 = cv2.dilate(erosion, element2, iterations = 3)
 
    # 7. 存储中间图片 
    cv2.imwrite("binary.png", binary)
    cv2.imwrite("dilation.png", dilation)
    cv2.imwrite("erosion.png", erosion)
    cv2.imwrite("dilation2.png", dilation2)
 
    return dilation2

In [18]:
def detect(img):
    # 1.  转化成灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # 2. 形态学变换的预处理，得到可以查找矩形的图片
    dilation = preprocess(gray)
 
    # 3. 查找和筛选文字区域
    region = findTextRegion(dilation)
 
    # 4. 用绿线画出这些找到的轮廓
    #for box in region:
        #cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
 
    #cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    #cv2.imshow("img", img)
 
    # 带轮廓的图片
    #cv2.imwrite("contours.png", img)
 
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return(region)

In [16]:
def my_ocr(img_path):
    ocr = PaddleOCR(use_angle_cls=True, lang='ch') # need to run only once to download and load model into memory
    result = ocr.ocr(img_path, det=False)
    
    for line in result:
        print(line)
        
    return result

In [6]:
def getcor(i,coordinates):
    
    x = [] ; y = []

    for temp in coordinates[i]:
        x.append(temp[1])
        y.append(temp[0])

    xmax = max(x) ; xmin = min(x)
    ymax = max(y) ; ymin = min(y)

    result = [xmin,xmax,ymin,ymax]
    
    return result

In [7]:
def crop(img,cor):
    
    result = img[cor[0]:cor[1],cor[2]:cor[3]]
    
    return result

In [28]:
def avg_distance(coordinate):
    a = 0
    for i in range (len(coordinate)-1):
        dis1 = getcor(i,coordinate)
        dis2 = getcor(i+1,coordinate)
        a = a + (dis2[0]+dis2[1] - dis1[0] - dis1[1])
    
    return (a/2)/len(coordinate)

In [82]:
#imagePath = 'C:/Users/HW/Documents/Python Scripts/OCR/OneDrive_2022-06-23/OCR Dataset/IMG_0845.JPG'
imagePath = 'deskewed.png'
if __name__ == '__main__':
    # 读取文件
    #imagePath = sys.argv[1]
    img = cv2.imread(imagePath)
    cor=detect(img)

coordinates=cor[::-1]
coordinates_int10=np.round(np.divide(coordinates,10))
numberofrows=len(coordinates)

columns=[['']]
numofparalines=0
dis = avg_distance(coordinates_int10)


for i in range(numberofrows):
    
    if i >= 1:
        tempcor1_int10 = getcor(i-1,coordinates_int10)
        tempcor2_int10 = getcor(i,coordinates_int10)
        
    
    if i==0:
        tempcor = getcor(i,coordinates)
        crop_img = crop(img,tempcor)
        #result = [pytesseract.image_to_string (crop_img, lang='chi_sim')]
        result = [my_ocr(crop_img)[0][0]]
        
        if result != ['']:
            columns.append(result)
            #blockindex.append(i)
        else:
            numofparalines+=1
            
    elif (tempcor2_int10[0]+tempcor2_int10[1])/2 - (tempcor1_int10[0]+tempcor1_int10[1])/2> 2*dis: # should be the same as the lower one
        #这边
        
        rowcor = [] # Use to take down the index of each row
        
        tempcor = getcor(i,coordinates)
        crop_img = crop(img,tempcor)
        #result = [pytesseract.image_to_string (crop_img, lang='chi_sim')]
        result = [my_ocr(crop_img)[0][0]]
        
        if result != ['']:
            columns.append(result)
            startrowindex = i
            rowcor.append(tempcor[0])
        else:
            numofparalines+=1
            

    else:
        numofparalines+=1
        tempcor1 = getcor(i-1,coordinates)
        tempcor2 = getcor(i,coordinates)
        
        tempcor = getcor(i,coordinates)
        crop_img = crop(img,tempcor)
        result = [my_ocr(crop_img)[0][0]]
        
        if result != ['']:
            columns[i-numofparalines].append(result)
            rowcor.append(tempcor[0])
        
        if i <= numberofrows-2:
            tempcor3_int10 = getcor(i+1,coordinates_int10)

            if (tempcor3_int10[0]+tempcor3_int10[1])/2 - (tempcor2_int10[0]+tempcor2_int10[1])/2> 2*dis: # should be the same as in the previous one
                
                if len(rowcor) != 0:
                    a,sortedrow = zip(*sorted(zip(rowcor,columns[-1])))
                    del columns[-1]

                    columns.append(list(sortedrow))
            
        #if tempcor2[2]>tempcor1[2]:
            
         #   if result != '':
          #      columns[i-numofparalines].append(result)
           #     blockindex.append(i)
            #else:
            #    numofparalines+=1
        
        #else: 
            
            #if result != '':
               #columns[i-numofparalines].append(result)
                #blockindex.append(i)
            

            
#for row in cloumns:
    
    
            
df=pd.DataFrame(columns)
df.fillna('',inplace=True)

# path="result/result.xlsx"
# if not os.path.exists(path):
#     os.makedirs(path)
df.to_excel('result.xlsx',index=False)

[2022/07/10 23:53:29] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_p

('上限市限份有限公司', 0.48763152956962585)
[2022/07/10 23:53:32] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_ty

('一', 0.49696657061576843)
[2022/07/10 23:53:35] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max'

('', 0.0)
[2022/07/10 23:53:38] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('本次条9调置143米', 0.533082127571106)
[2022/07/10 23:53:41] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_typ

...

('199999', 0.9714624285697937)
[2022/07/10 23:55:55] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='

('125313', 0.9944664835929871)
[2022/07/10 23:55:58] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='

('', 0.0)
[2022/07/10 23:56:01] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('', 0.0)
[2022/07/10 23:56:04] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('打印机构号：237330', 0.9849457144737244)
[2022/07/10 23:56:07] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_

In [41]:
blockindex[195]

217

In [44]:
blockindex[-1]

217

In [59]:
list1 = ['1.','2.']
list2 = ['1','2']

In [62]:
a,b = zip(*list(zip(list1,list2)))

In [64]:
list(a)

['1.', '2.']

In [50]:
del list1[-1]

In [51]:
list1

['1.']

In [66]:
columns

[['', '中国农业银行银行'],
 ('卡/活期存折交易明细清单',),
 ['上限市限份有限公司', '序号：000', '账号户名：6213360369900632171陈勇', '一'],
 ('币种：人民币',),
 ('江口县支行',),
 ['业子专用章']]

In [67]:
rowcor

[2298]

In [69]:
columns

[['', ['中国农业银行银行']],
 ('卡/活期存折交易明细清单',),
 ['上限市限份有限公司', ['序号：000'], ['账号户名：6213360369900632171陈勇'], ['一']],
 ('币种：人民币',),
 ['江口县支行']]

In [71]:
columns

[['', ['中国农业银行银行']],
 ('卡/活期存折交易明细清单',),
 ['上限市限份有限公司', ['序号：000'], ['账号户名：6213360369900632171陈勇'], ['一']],
 ('币种：人民币',),
 ('江口县支行',),
 ['业子专用章']]

In [74]:
df=pd.DataFrame(columns)
df.fillna('',inplace=True)

# path="result/result.xlsx"
# if not os.path.exists(path):
#     os.makedirs(path)
df.to_excel('result2.xlsx',index=False)

In [76]:
columns

[['', ['中国农业银行银行']],
 ['卡/活期存折交易明细清单'],
 ['上限市限份有限公司', ['序号：000'], ['账号户名：6213360369900632171陈勇'], ['一']],
 ['币种：人民币'],
 ['江口县支行', ['起止口期：20200514-20200814'], ['本次条9调置143米']],
 ['业子专用章', ['短润婆'], ['山志号'], ['时间'], ['口期']],
 ['交易金额',
  ['EPAY'],
  ['509999'],
  ['12267.04'],
  ['12000.00'],
  ['代付'],
  ['366049660'],
  ['133820'],
  ['20200518']],
 ['附言'],
 ['24300133财付通支付科技有限公司NG2020018200057843202429011101微信零钱据现',
  ['237330'],
  ['7267.04'],
  ['5000.00'],
  ['现支'],
  ['383211970'],
  ['135807'],
  ['20200518']],
 ['ATMP',
  ['237330'],
  ['2267.04'],
  ['5000.00'],
  ['现支'],
  ['383698692'],
  ['135900'],
  ['20200518']],
 ['ATMP',
  ['199999'],
  ['2252.01'],
  ['15.00'],
  ['支付宝'],
  ['196664667'],
  ['083747'],
  ['20200519']],
 ['EPAY'],
 ['215500690A2020051920878852080277220311500广州驹计算机科技有限公司',
  ['199999'],
  ['-110.30'],
  ['财付道'],
  ['587489691'],
  ['191342'],
  ['20200520']],
 ['EPAY'],
 ['243300133NA2020052029110065025471210300109江口县太顺发时代超市',
  ['199999'],
  ['1641.74'],
 

In [77]:
row = []

In [78]:
len(row)

0

In [81]:
df=pd.DataFrame(columns)
df.fillna('',inplace=True)

# path="result/result.xlsx"
# if not os.path.exists(path):
#     os.makedirs(path)
df.to_excel('result3.xlsx',index=False)

In [83]:
a = img[100:200,200:400]
cv2.imwrite('test.png',a)

True